## Mount drive

In [ ]:
#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Libraries

In [ ]:
! pip install git+https://github.com/openai/whisper.git -q
! pip install gradio -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.4

In [ ]:
import shutil
import whisper
import gradio as gr 
import time

model = whisper.load_model("base")


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 59.0MiB/s]


In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
!pip show gradio

Name: gradio
Version: 3.28.3
Summary: Python library for easily interacting with trained machine learning models
Home-page: 
Author: 
Author-email: Abubakar Abid <team@gradio.app>, Ali Abid <team@gradio.app>, Ali Abdalla <team@gradio.app>, Dawood Khan <team@gradio.app>, Ahsen Khaliq <team@gradio.app>, Pete Allen <team@gradio.app>, Ömer Faruk Özdemir <team@gradio.app>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiofiles, aiohttp, altair, fastapi, ffmpy, gradio-client, httpx, huggingface-hub, jinja2, markdown-it-py, markupsafe, matplotlib, mdit-py-plugins, numpy, orjson, pandas, pillow, pydantic, pydub, pygments, python-multipart, pyyaml, requests, semantic-version, typing-extensions, uvicorn, websockets
Required-by: 


In [ ]:
# from IPython.display import Audio
# Audio("/content/pokkiri-message-3920.mp3")

In [ ]:
%%time
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("/content/drive/MyDrive/Whisper/audio/206Elissa.mp3")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

Detected language: en
Welcome to the Huberman Lab podcast where we discuss science and science-based tools for everyday life. I'm Andrew Huberman and I'm a professor of neurobiology and Ophthalmology at Stanford School of Medicine. Today my guest is Dr. Alyssa Eppel. Dr. Eppel is a professor of psychiatry and behavioral sciences at the University of California, San Francisco. She is also the director of the Center on Aging, Metabolism and Emotions.
CPU times: user 4.73 s, sys: 1.88 s, total: 6.61 s
Wall time: 23.6 s


In [ ]:
audio.shape

(480000,)

In [ ]:
# 
def transcribe(audio):
    
    #time.sleep(3)
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

    # decode the audio
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)

    # # Copy the log.csv file to the output folder

    # shutil.copy("/content/flagged/log.csv", "/content/drive/MyDrive/Whisper/output/log.csv")
    return result.text


## Batch mp3 microphone, lonely mp3

In [ ]:
# Looks ok getting audio files from filewraper and delivering them one by one to transcribe
def my_function(files):

  temp=''
  filenames = [file.name for file in files]
  for audio_file in filenames:
    print(audio_file)
    result=transcribe(audio_file)
    temp=temp+result
  return temp

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("Transcribe with Whisper")
    with gr.Tab("transcribe mp3"):
        mp3_input = gr.inputs.Audio(label="Upload Audio File (.mp3)", type="filepath")
        # mp3_input = gr.inputs.File(file_count="multiple",label="Input Files")
        
        text_output1 = gr.Textbox()
        text_button = gr.Button("Transcribe")
    with gr.Tab("transcribe in Batch  mp3 files "):
        # mp3_input = gr.inputs.Audio(label="Upload Audio File (.mp3)", type="filepath")
        mp3_batch = gr.inputs.File(file_count="multiple",label="Input Files")
        
        text_output3 = gr.Textbox()
        text_button3 = gr.Button("Transcribe")

    with gr.Tab("transcribe mike"):
        with gr.Row():
            mike_input = gr.inputs.Audio(source="microphone", type="filepath")
            text_output2 = gr.Textbox()
        image_button = gr.Button("Transcribe")

    # with gr.Accordion("Open for More!"):
    #     gr.Markdown("Look at me...")

    text_button.click(transcribe, inputs=mp3_input, outputs=text_output1)
    image_button.click(transcribe, inputs=mike_input, outputs=text_output2)
    text_button3.click(my_function, inputs=mp3_batch, outputs=text_output3)

demo.launch(share=True, debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:321: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:349: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `keep_filename` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a87b5781597778fdbd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


/tmp/gradio/161d3c43e7bc0fd80c90b71895a8cbe4b61d6981/ryszard.mp3
Detected language: pl
/tmp/gradio/198fc7818d4fa6830a45704384f1cad49ed182d0/ryszard_001.mp3
Detected language: pl
/tmp/gradio/55b9ce492d1a86b6610ec1238ae45e6c9a8ef967/ryszard_002.mp3
Detected language: en
/tmp/gradio/5c5800bd47288bf45f6959fe29a6f7074b18d922/huberman_002.mp3
Detected language: en
/tmp/gradio/c03249a9e5aff171c5ad8a9772d8997a1a0b07fd/huberman_006.mp3
Detected language: en
/tmp/gradio/bce2fe47e2f3280efe7c38e7f395597de10f0013/huberman_008.mp3
Detected language: en
/tmp/gradio/578e6ad4ea9fdd64ecc656d9463ca360a3451aa0/huberman_000.mp3
Detected language: en
/tmp/gradio/36c27d11207dea9536a9df47e552c9938b5cf6a0/huberman_001.mp3
Detected language: en
/tmp/gradio/5c5800bd47288bf45f6959fe29a6f7074b18d922/huberman_002.mp3
Detected language: en
/tmp/gradio/b3dd7239d815de08b6c4e09686f14c31f4390ec7/huberman_003.mp3
Detected language: en
/tmp/gradio/611483d6a3465de2c2feceb835373eb9fd8b99d0/huberman_004.mp3
Detected language